In [1]:
!pip install snowflake-connector-python


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import snowflake.connector
import logging

SNOWFLAKE_USER = "your_username_here"        
SNOWFLAKE_PASSWORD = "your_password_here"      
SNOWFLAKE_ACCOUNT = "your_account_identifier_here"    
SNOWFLAKE_WAREHOUSE = "your_warehouse_here"         
SNOWFLAKE_ROLE = "your_role_here" 

In [ ]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    role=SNOWFLAKE_ROLE,
    database="your_database_here",
    schema="your_schema_here"  
)

cur = conn.cursor()

try:
    print("Creating tables with PERFECT column match to your real CSVs...")

    # dimDate.csv → exactly 5 columns, fips first
    cur.execute("""
        CREATE OR REPLACE TABLE dimDate (
            fips        NUMBER,
            date        STRING,     -- '2021-03-07' format
            month       NUMBER,
            year        NUMBER,
            day_of_week NUMBER
        )
    """)

    # dimHospital.csv → exactly 9 columns (note: header spells it "longtitude")
    cur.execute("""
        CREATE OR REPLACE TABLE dimHospital (
            fips          FLOAT,
            state_name    STRING,
            latitude      FLOAT,
            longitude     FLOAT,    -- fixed spelling, but still loads correctly (position-based)
            hq_address    STRING,
            hospital_name STRING,
            hospital_type STRING,
            hq_city       STRING,
            hq_state      STRING
        )
    """)

    # factCovid.csv → exactly 13 columns, no index column
    cur.execute("""
        CREATE OR REPLACE TABLE factCovid (
            fips                    FLOAT,
            province_state          STRING,
            country_region          STRING,
            confirmed               FLOAT,
            deaths                  FLOAT,
            recovered               FLOAT,
            active                  FLOAT,
            date                    NUMBER,   -- 20210307 format
            positive                INTEGER,
            negative                FLOAT,
            hospitalizedcurrently   FLOAT,
            hospitalized            FLOAT,
            hospitalizeddischarged  FLOAT
        )
    """)

    cur.execute("""
        CREATE OR REPLACE TABLE dimRegion (
            fips           FLOAT,     -- can be null/empty
            province_state STRING,
            country_region STRING,
            latitude       FLOAT,
            longitude      FLOAT,
            county         STRING,
            state          STRING
        )
    """)

    print("Loading all 4 files from the correct stage path...")

    ff = """
        FILE_FORMAT = (
            TYPE = CSV 
            SKIP_HEADER = 1 
            FIELD_DELIMITER = ',' 
        ) 
        ON_ERROR = 'CONTINUE'
    """

    cur.execute(f"COPY INTO dimDate     FROM @covid_s3_stage/dwresults/dimDate.csv     {ff}")
    cur.execute(f"COPY INTO dimHospital FROM @covid_s3_stage/dwresults/dimHospital.csv {ff}")
    cur.execute(f"COPY INTO factCovid   FROM @covid_s3_stage/dwresults/factCovid.csv   {ff}")
    cur.execute(f"COPY INTO dimRegion   FROM @covid_s3_stage/dwresults/dimRegion.csv   {ff}")

    print("Done! Final Row counts:")
    for table in ["dimDate", "dimHospital", "factCovid", "dimRegion"]:
        cur.execute(f"SELECT COUNT(*) FROM {table}")
        print(f"{table}: {cur.fetchone()[0]:,} rows")
    
except Exception as e:
    print("Error:", e)
    raise
finally:
    cur.close()
    conn.close()

Creating tables with PERFECT column match to your real CSVs...
Loading all 4 files from the correct stage path...
Done! Final Row counts:
dimDate: 10,187 rows
dimHospital: 6,637 rows
factCovid: 1,092 rows
dimRegion: 1,017,538 rows
